In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams['font.sans-serif'] = ['SimHei']
rcParams['axes.unicode_minus'] = False

# 假设从Excel中读取的次品率已更新
def load_defective_rates():
    df = pd.read_excel('问题3零配件半成品成品更新次品率.xlsx')
    return df.set_index('项目')['更新次品率'].to_dict()

def calculate_expected_cost(components, semi_products, final_product, detect_components, detect_semi, detect_final, dismantle_semi, dismantle_final):
    total_cost = 0

    # 零配件成本
    for i, (comp_name, comp_data) in enumerate(components.items()):
        cost = comp_data['购买单价']
        if detect_components[i]:
            cost += comp_data['检测成本']
        total_cost += cost

    # 半成品成本及拆解费用
    for i, (semi_name, semi_data) in enumerate(semi_products.items()):
        cost = semi_data['装配成本']
        if detect_semi[i]:
            cost += semi_data['检测成本']
        total_cost += cost

        if dismantle_semi[i]:
            total_cost += semi_data['拆解费用']

    # 成品成本
    final_cost = final_product['成品']['装配成本']
    if detect_final:
        final_cost += final_product['成品']['检测成本']
    total_cost += final_cost

    # 换货损失
    defective_rate = final_product['成品']['次品率']
    exchange_loss = final_product['成品']['调换损失'] * defective_rate
    total_cost += exchange_loss

    # 成品拆解费用
    if dismantle_final:
        total_cost += final_product['成品']['拆解费用']

    return total_cost

def generate_strategy_description(detect_components, detect_semi, detect_final, dismantle_semi, dismantle_final):
    description = ""
    for i, detect in enumerate(detect_components):
        description += f"检测零配件 {i + 1}，" if detect else f"不检测零配件 {i + 1}，"
    for i, detect in enumerate(detect_semi):
        description += f"检测半成品 {i + 1}，" if detect else f"不检测半成品 {i + 1}，"
    description += "检测成品，" if detect_final else "不检测成品，"
    for i, dismantle in enumerate(dismantle_semi):
        description += f"拆解半成品 {i + 1}，" if dismantle else f"不拆解半成品 {i + 1}，"
    description += "拆解不合格成品" if dismantle_final else "不拆解不合格成品"
    return description

def fitness_function(chromosome, components, semi_products, final_product):
    detect_components = chromosome[:8]
    detect_semi = chromosome[8:11]
    detect_final = chromosome[11]
    dismantle_semi = chromosome[12:15]
    dismantle_final = chromosome[15]

    cost = calculate_expected_cost(components, semi_products, final_product, detect_components, detect_semi, detect_final, dismantle_semi, dismantle_final)
    return cost

def initialize_population(population_size, num_genes):
    return np.random.randint(2, size=(population_size, num_genes))

def selection(population, fitness_values, num_parents):
    parents = np.empty((num_parents, population.shape[1]), dtype=int)
    for i in range(num_parents):
        min_fitness_idx = np.argmin(fitness_values)
        parents[i, :] = population[min_fitness_idx, :]
        fitness_values[min_fitness_idx] = np.inf
    return parents

def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size, dtype=int)
    crossover_point = np.uint8(offspring_size[1] / 2)

    for k in range(offspring_size[0]):
        parent1_idx = k % parents.shape[0]
        parent2_idx = (k + 1) % parents.shape[0]
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

def mutation(offspring_crossover, mutation_rate):
    for idx in range(offspring_crossover.shape[0]):
        if np.random.rand() < mutation_rate:
            random_gene_idx = np.random.randint(offspring_crossover.shape[1])
            offspring_crossover[idx, random_gene_idx] = 1 - offspring_crossover[idx, random_gene_idx]
    return offspring_crossover

def genetic_algorithm(components, semi_products, final_product, population_size=1000, num_generations=100, mutation_rate=0.01):
    num_genes = 16  # 8检测零配件 + 3检测半成品 + 1检测成品 + 3半成品拆解 + 1成品拆解
    population = initialize_population(population_size, num_genes)
    results = []

    for generation in range(num_generations):
        fitness_values = np.array([fitness_function(chromosome, components, semi_products, final_product) for chromosome in population])
        parents = selection(population, fitness_values, population_size // 2)
        offspring_crossover = crossover(parents, offspring_size=(population_size - parents.shape[0], num_genes))
        offspring_mutation = mutation(offspring_crossover, mutation_rate)
        population[0:parents.shape[0], :] = parents
        population[parents.shape[0]:, :] = offspring_mutation

        # 记录每一代的最优策略
        best_chromosome = population[np.argmin(fitness_values)]
        best_cost = fitness_function(best_chromosome, components, semi_products, final_product)
        best_strategy_description = generate_strategy_description(best_chromosome[:8], best_chromosome[8:11], best_chromosome[11], best_chromosome[12:15], best_chromosome[15])
        results.append([generation + 1, best_strategy_description, best_cost])

    # 找到最优策略
    best_chromosome = population[np.argmin(fitness_values)]
    best_cost = fitness_function(best_chromosome, components, semi_products, final_product)
    best_strategy_description = generate_strategy_description(best_chromosome[:8], best_chromosome[8:11], best_chromosome[11], best_chromosome[12:15], best_chromosome[15])

    return best_strategy_description, best_cost, results

# 使用遗传算法求解最优策略
best_strategy_description, best_cost, results = genetic_algorithm(components, semi_products, final_product)

# 将结果保存到Excel文件中
df = pd.DataFrame(results, columns=['策略编号', '策略描述', '总成本'])
df.to_excel('B4_2策略结果.xlsx', index=False)
print("所有策略已经保存到 'B4_2策略结果.xlsx' 文件中。")

print(f"最优策略: {best_strategy_description}")
print(f"最优策略总成本: {best_cost:.2f}")

# 可视化结果
plt.figure(figsize=(10, 6))
plt.plot(df['策略编号'], df['总成本'], marker='o', linestyle='-', color='blue', label="总成本")
plt.xlabel("策略编号")
plt.ylabel("总成本")
plt.title("不同策略下的总成本比较")
plt.grid(True)
plt.legend
